
# Event Agent with LangGraph + OpenAI + LangSmith

This notebook integrates:
- **LangGraph** for workflow orchestration
- **OpenAI** LLM (real API calls, using `OPENAI_API_KEY` from your `.env`)
- **LangSmith** tracing (using `LANGSMITH_API_KEY` from your `.env`)

## Setup
1. Create a `.env` file in the same directory with:
```
OPENAI_API_KEY=sk-...
LANGSMITH_API_KEY=ls-...
# Optional overrides:
# OPENAI_MODEL=gpt-4o-mini
# LANGSMITH_PROJECT=Panel Monitoring Agent
```
2. Run all cells below.


In [4]:

import os, time, uuid
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
from langsmith import Client
from langsmith.run_helpers import trace

load_dotenv(find_dotenv(), override=False)

# Make sure no sticky session is forcing collisions
os.environ.pop("LANGSMITH_SESSION", None)

# Tracing + project
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING", "true")
PROJECT = os.getenv("LANGSMITH_PROJECT") or f"Panel Monitoring Agent ({datetime.now().strftime('%Y-%m-%d %H:%M')})"
os.environ["LANGSMITH_PROJECT"] = PROJECT

# Ensure project exists *before* tracing
c = Client()
c.create_project(PROJECT, upsert=True)

print("Endpoint:", c.api_url)
print("Project:", PROJECT)

# Minimal traced ping so the project appears immediately
with trace(
    name=f"sanity-ping-{uuid.uuid4().hex[:6]}",
    project_name=PROJECT,
    tags=["notebook"]
):
    pass

print("✅ Sent traced ping")


LangSmithConflictError: Conflict for https://api.smith.langchain.com/sessions. HTTPError('409 Client Error: Conflict for url: https://api.smith.langchain.com/sessions', '{"detail":"Session already exists."}')

## Define Graph State and Nodes (with OpenAI + LangSmith)

In [5]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
import json, random, os
from langsmith import traceable
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langsmith.run_helpers import trace

class GraphState(TypedDict):
    event_data: str
    signals: dict
    action: str
    log_entry: str

# Structured schema for LLM output
class Signals(BaseModel):
    suspicious_signup: bool = Field(..., description="True if the event is suspicious")
    normal_signup: bool = Field(..., description="True if the event is normal")

# LLM client (auto-traced to LangSmith)
llm = ChatOpenAI(
    model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
    temperature=0,
)
with trace(name="sanity-ping", project_name=os.environ["LANGSMITH_PROJECT"], tags=["notebook"]):
    _ = llm.invoke("ping")

print("Sent a traced run. Refresh Studio → Projects.")
structured = llm.with_structured_output(Signals)

@traceable(project_name=os.environ["LANGSMITH_PROJECT"])
def user_event_node(state: GraphState) -> GraphState:
    print("--- 1. User Event Node ---")
    return {"event_data": state.get("event_data", ""), "signals": {}, "action": "", "log_entry": ""}

def signal_evaluation_node(state: GraphState) -> GraphState:
    print("--- 2. Signal Evaluation Node: LLM classifying event ---")
    event = state.get("event_data", "")
    prompt = (
        "Decide if a signup event is suspicious. Suspicious only if there are clear fraud cues "
        "(e.g., admits to being a bot, spam/test text, contradictory claims, obviously fake identity, "
        "or explicit words like 'fraud', 'scam', 'bot', 'test', 'fake'). High income or foreign location "
        "alone is NOT suspicious.\n\n"
        "Return booleans so exactly one of 'suspicious_signup' or 'normal_signup' is true, plus a "
        "'confidence' 0..1 and a short 'reason'.\n\n"
        "Examples:\n"
        "- 'I am a 22 year old Director making $200k' -> suspicious_signup:true (contradictory age/role), reason why.\n"
        "- 'I am a 35 year old teacher making $50k' -> normal_signup:true, reason why.\n"
        "- 'this is a test bot signup' -> suspicious_signup:true, reason why.\n\n"
        f"Event: {event}"
    )
    try:
        result: Signals = structured.invoke(prompt)
        signals = result.model_dump()
        if signals["suspicious_signup"] == signals["normal_signup"]:
            heur = "suspicious" in event.lower() or ("director" in event.lower() and "22" in event.lower())
            signals = {"suspicious_signup": heur, "normal_signup": not heur}
    except Exception as e:
        print("LLM error, fallback heuristic:", e)
        heur = "suspicious" in event.lower() or ("director" in event.lower() and "22" in event.lower())
        signals = {"suspicious_signup": heur, "normal_signup": not heur}
    return {"signals": signals}

def action_decision_node(state: GraphState) -> GraphState:
    print("--- 3. Action Decision Node ---")
    signals = state.get("signals", {})
    action = "no_action"
    if signals.get("suspicious_signup"):
        action = "remove_account" if random.random() > 0.5 else "hold_account"
    return {"action": action}

def logging_node(state: GraphState) -> GraphState:
    print("--- 4. Logging Node ---")
    log_summary = {
        "event": state.get("event_data", "N/A"),
        "signals_detected": state.get("signals", {}),
        "final_action": state.get("action", "N/A")
    }
    log_entry = json.dumps(log_summary, indent=2)
    print("\n--- Final Log Entry ---\n" + log_entry)
    return {"log_entry": log_entry}


Sent a traced run. Refresh Studio → Projects.


## Assemble and Compile the Graph

In [6]:
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(GraphState)
workflow.add_node("user_event_node", user_event_node)
workflow.add_node("signal_evaluation_node", signal_evaluation_node)
workflow.add_node("action_decision_node", action_decision_node)
workflow.add_node("logging_node", logging_node)

workflow.add_edge(START, "user_event_node")
workflow.add_edge("user_event_node", "signal_evaluation_node")
workflow.add_edge("signal_evaluation_node", "action_decision_node")
workflow.add_edge("action_decision_node", "logging_node")
workflow.add_edge("logging_node", END)

app = workflow.compile()
print("✅ Graph compiled")


✅ Graph compiled


## Test the Graph

In [7]:
import os
from uuid import uuid4
from langsmith.run_helpers import trace

PROJECT = os.getenv("LANGSMITH_PROJECT", "Panel Monitoring Agent")
RUN_NAME = f"Event Agent Run {uuid4().hex[:6]}"

with trace(name=RUN_NAME, project_name=PROJECT):  # no session_name -> avoids 409s
    final_state = app.invoke({"event_data": "NEW SIGNUP: ..."})
    print("==========================================================")
    print("TESTING SCENARIO 1: SUSPICIOUS SIGNUP")
    print("==========================================================")
    suspicious_event = "NEW SIGNUP: I am a 22 year old Director making $200,000 per year."
    final_state_1 = app.invoke({"event_data": suspicious_event})

    print("\n\n==========================================================")
    print("TESTING SCENARIO 2: NORMAL SIGNUP")
    print("==========================================================")
    normal_event = "NEW SIGNUP: I am a 35 year old teacher making $50,000 per year."
    final_state_2 = app.invoke({"event_data": normal_event})


--- 1. User Event Node ---
--- 2. Signal Evaluation Node: LLM classifying event ---
--- 3. Action Decision Node ---
--- 4. Logging Node ---

--- Final Log Entry ---
{
  "event": "NEW SIGNUP: ...",
  "signals_detected": {
    "suspicious_signup": true,
    "normal_signup": false
  },
  "final_action": "remove_account"
}
TESTING SCENARIO 1: SUSPICIOUS SIGNUP
--- 1. User Event Node ---
--- 2. Signal Evaluation Node: LLM classifying event ---
--- 3. Action Decision Node ---
--- 4. Logging Node ---

--- Final Log Entry ---
{
  "event": "NEW SIGNUP: I am a 22 year old Director making $200,000 per year.",
  "signals_detected": {
    "suspicious_signup": true,
    "normal_signup": false
  },
  "final_action": "remove_account"
}


TESTING SCENARIO 2: NORMAL SIGNUP
--- 1. User Event Node ---
--- 2. Signal Evaluation Node: LLM classifying event ---
--- 3. Action Decision Node ---
--- 4. Logging Node ---

--- Final Log Entry ---
{
  "event": "NEW SIGNUP: I am a 35 year old teacher making $50,000 p

In [9]:
event = input("Type a signup event and press Enter:\n> ")
final_state = app.invoke({"event_data": event})

print("=== RESULTS ===")
print("Signals:", final_state.get("signals"))
print("Action :", final_state.get("action"))
print("Log    :\n", final_state.get("log_entry"))


Type a signup event and press Enter:
>  i hear from youtube, i am a cook in new york city, i sighed up last month, i have 5 types of pets. i have 6 dogs and 10 cats.


--- 1. User Event Node ---
--- 2. Signal Evaluation Node: LLM classifying event ---
--- 3. Action Decision Node ---
--- 4. Logging Node ---

--- Final Log Entry ---
{
  "event": "i hear from youtube, i am a cook in new york city, i sighed up last month, i have 5 types of pets. i have 6 dogs and 10 cats.",
  "signals_detected": {
    "suspicious_signup": false,
    "normal_signup": true
  },
  "final_action": "no_action"
}
=== RESULTS ===
Signals: {'suspicious_signup': False, 'normal_signup': True}
Action : no_action
Log    :
 {
  "event": "i hear from youtube, i am a cook in new york city, i sighed up last month, i have 5 types of pets. i have 6 dogs and 10 cats.",
  "signals_detected": {
    "suspicious_signup": false,
    "normal_signup": true
  },
  "final_action": "no_action"
}
